In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
data = pd.read_csv("train.csv",encoding = "big5")
variables = 9

In [2]:
def RMSE(Weight,Bias,X,Y,TDataMonth,TDataLeng):
    cost = 0.0
    for i in range(TDataMonth):
        for j in range(TDataLeng):
            cost = cost + (Y[i*TDataLeng+j] - Bias-np.dot(Weight,X[i,j,0:variables]))**2.0
    error = (cost/(TDataMonth*TDataLeng))**(0.5)
    return error

In [3]:
def fscaling(V):
    FS = (V-np.mean(V))/(np.max(V)-np.min(V))
    return FS

In [4]:
def Tfscaling(V,TR):
    FS = (V-np.mean(TR))/(np.max(TR)-np.min(TR))
    return FS

In [5]:
def TFprocess(data,S):
    V = data[data["測項"]==S]
    V = V.drop(['日期','測站','測項'],axis = 1)
    V = np.array(V,float)
    v = np.reshape(V,(12,480))
    datarow = len(v)
    datacol = len(v[0])
    DSV = np.zeros((datarow,datacol-9,9),float)
    for i in range(0,datarow):
        for j in range(0,datacol-9):
            for k in range(0,9):
                DSV[i,j,k] = v[i,j+k]
    return DSV

In [6]:
def TLprocess(data,S):
    V = data[data["測項"]==S]
    V = V.drop(['日期','測站','測項'],axis = 1)
    V = np.array(V,float)
    v = np.reshape(V,(12,480))
    datarow = len(v)
    datacol = len(v[0])
    DSV = np.zeros(datarow*(datacol-9),float)
    for i in range(0,datarow):
        for j in range(0,datacol-9):
            DSV[i*(datacol-9)+j] = v[i,j+9]      #target
    return DSV

In [7]:
def StackData(X,S,data):
    if len(X) == 0:
        X = TFprocess(data,S)
        return X
    else:
        V = TFprocess(data,S)
        X = np.dstack([X,V])
        return X

In [8]:
def TStackData(Xt,S,tdata):
    Vt = tdata[tdata['測項']==S]
    Vt=Vt.drop(['1','測項'],axis = 1)
    Vt = np.array(Vt,float)
    TR = data[data["測項"]==S]
    TR = TR.drop(['日期','測站','測項'],axis = 1)
    TR = np.array(TR,float)
    if len(Xt) == 0:
        return Vt
    else:
        Xt = np.hstack([Xt,Vt])
        return Xt

In [9]:

X = []
X = StackData(X,"PM2.5",data)
X = np.dstack([X,X**2])
X = np.dstack([X,X**3])
X = StackData(X,"PM10",data)
#print(X)
Y = TLprocess(data,"PM2.5")
TDataDimen = variables#len(X[0,0])
TDataLeng = len(X[0])
TDataMonth = len(X)
#print(Y)

In [10]:
# train model y = b+wx+Lw^2
Bias = 10.0
#Xk is {pm_25 if k = 0~8} {pm_10 if k = 9~17} {so2 if k = 18~26} {no2 if k = 27~35}
#{nox if k = 36~44} {o3 if k = 45~53}
Weight = np.zeros(TDataDimen,float)

lr = 1
itera = 10000
lamda = 0.000

B_lr = 0.0
W_lr = np.zeros(TDataDimen,float)
error_his = []
times = []

for it in range(itera):
    B_grad = 0.0
    W_grad = np.zeros(TDataDimen,float)
    for i in range(TDataMonth):
        for j in range(0,TDataLeng):
            L = Y[i*TDataLeng+j]-(Bias+np.dot(X[i,j,0:variables],Weight))
            B_grad = B_grad - 2.0*L*1.0
            W_grad = W_grad - 2.0*L*X[i,j,0:variables] + lamda*2.0*Weight

    B_lr = B_lr+B_grad**2
    W_lr = W_lr+W_grad**2
    

    # Update parameters
    Bias = Bias - lr/np.sqrt(B_lr) * B_grad
    Weight = Weight - lr/np.sqrt(W_lr)*W_grad
    #error=RMSE(Weight,Bias,X,Y,TDataMonth,TDataLeng)
    #error_his.append(error)
    #times.append(it)
#plt.scatter(times[2:],error_his[2:])
#plt.show()

In [11]:
print('Bias :',Bias)
print('Weight:',Weight)

Bias : 1.73371187025
Weight: [ -3.31918231e-02  -2.59165171e-02   2.18117059e-01  -2.39634842e-01
  -5.44907633e-02   5.30830663e-01  -5.72083357e-01   8.04994242e-04
   1.09459211e+00]


In [12]:
print("Root Mean Square Error:",RMSE(Weight,Bias,X,Y,TDataMonth,TDataLeng))

Root Mean Square Error: 6.12302152223


In [13]:
tdata = pd.read_csv("test_X.csv",encoding = "big5",names = ['1','測項','3','4','5','6','7','8','9','10','11'])

PM_25t = tdata[tdata['測項']=="PM2.5"]
index_temp = np.array(PM_25t)
index = index_temp[:,0]

Xt = []
Xt = TStackData(Xt,"PM2.5",tdata)
Xt = np.hstack([Xt,Xt**2])
Xt = np.hstack([Xt,Xt**3])
Xt = TStackData(Xt,"PM10",tdata)


TrDataDimen = len(Xt[0])
TrDataLeng = len(Xt)


In [14]:
result = np.zeros((TrDataLeng),float)
for i in range(TrDataLeng):
    result[i] = Bias + np.dot(Weight,Xt[i,0:variables])
    

In [15]:
final = [index,result]
final = np.transpose(final)
Final = pd.DataFrame(final,columns = ['id','value'] )
print(Final)
Final.to_csv("hw1.csv",index = False)

         id     value
0      id_0   26.5567
1      id_1   61.5622
2      id_2   20.4769
3      id_3   28.5279
4      id_4   10.3229
5      id_5   36.1825
6      id_6   39.4747
7      id_7   16.5593
8      id_8   47.8124
9      id_9   33.2884
10    id_10   38.8693
11    id_11   58.2873
12    id_12   36.5647
13    id_13   41.4133
14    id_14   26.6085
15    id_15      12.7
16    id_16   52.6848
17    id_17   22.4939
18    id_18   19.4213
19    id_19   19.7488
20    id_20   14.9632
21    id_21   28.3559
22    id_22    36.771
23    id_23    9.3862
24    id_24   57.8903
25    id_25   44.2575
26    id_26   28.3827
27    id_27   9.39623
28    id_28   29.0928
29    id_29   35.6332
..      ...       ...
210  id_210    28.202
211  id_211   72.2947
212  id_212   20.6258
213  id_213   115.858
214  id_214   39.9826
215  id_215    83.673
216  id_216   92.6621
217  id_217   70.6637
218  id_218   28.8915
219  id_219   18.8906
220  id_220     31.82
221  id_221   55.3367
222  id_222   28.7885
223  id_22